<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Параметры-для-запуска" data-toc-modified-id="Параметры-для-запуска-1">Параметры для запуска</a></span></li><li><span><a href="#Чтение-данных" data-toc-modified-id="Чтение-данных-2">Чтение данных</a></span></li></ul></div>

In [1]:
import os
import sys

In [2]:
from tqdm.auto import tqdm

In [3]:
import napari

In [4]:
import scipy
from scipy import ndimage as ndi
import numpy as np
import networkx as nx

from pathlib import Path

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import pickle
from astrocyte import Astrocyte

In [7]:
import ccdb
import astromorpho as astro

Can't load imreg package, affine and homography registrations won't work


/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/tifffile.py:7261: UserWarning: No module named 'imfun.external._tifffile'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/czifile.py:146: UserWarning: ImportError: No module named '_czifile'. Decoding of JXR and JPEG encoded images will not be available. Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  warnings.warn(
2022-08-31 16:30:03.727136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/levtg/astro-morpho/venv/lib/python3.10/site-packages/cv2/../../lib64:/home/levtg/v3d_external/v3d_main/common_lib/lib/
2022-08-31 16:30:03.727157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dler

In [8]:
import graph_utils as gu  
import visualization as vis

In [9]:
from networx2napari import draw_edges, draw_nodes

# Параметры для запуска

In [10]:
if os.path.exists('/home/levtg/astro-morpho'):
#     data_dir = '/home/levtg/astro-morpho/data/graphs'
    data_dir = '/home/levtg/astro-morpho/src/output/'
else:
    print("Dont know where to look for the data")

output_dir = './output/'

filename = '4wk-ly9-raw.pic-sequential_graph.pickle' # Octopus

VERBOSE = True

# Set false to start from console
HANDY = True

# Set true to save output
OUT = False

# Чтение данных

In [82]:
filepath = Path(data_dir).joinpath(filename)
filepath

PosixPath('/home/levtg/astro-morpho/src/output/4wk-ly9-raw.pic-sequential_graph.pickle')

In [108]:
%time data = pickle.load(open(filepath, 'rb'))

CPU times: user 40.9 ms, sys: 36.9 ms, total: 77.8 ms
Wall time: 76.3 ms


In [84]:
w =  napari.view_image(data.image, ndisplay=3, opacity=0.5)
vis.view_graph_as_colored_image(data.graph, data.parameters['shape'], w);

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [85]:
roots = gu.get_roots(data.graph)
roots

{(18, 153, 172),
 (19, 159, 170),
 (19, 159, 171),
 (20, 132, 173),
 (20, 159, 169),
 (20, 160, 170),
 (21, 144, 170),
 (21, 156, 167),
 (21, 158, 169),
 (21, 159, 170),
 (22, 130, 176),
 (22, 145, 170),
 (22, 148, 167),
 (22, 149, 166),
 (22, 167, 195),
 (23, 135, 194),
 (23, 147, 169),
 (23, 151, 167),
 (27, 170, 190),
 (27, 173, 187),
 (28, 170, 189),
 (29, 161, 183),
 (29, 170, 182),
 (30, 166, 181)}

In [86]:
branches = {}
for root in roots:
    branches[root] = gu.filter_graph(data.graph, lambda node: node['root'] == root)

In [87]:
import itertools as itt

w = napari.view_image(data.image, opacity=0.5)
pos = {node: node for node in data.graph.nodes}

colors = ['blue', 'red', 'yellow', 'cyan', 'green', 'magenta', 'bop orange']
for branch, color in zip(branches.values(), itt.cycle(colors)):
    print(branch)
    img = np.zeros(data.image.shape)
    points = draw_nodes(pos, branch.nodes)
    img[points[:, 0], points[:, 1], points[:, 2]] = 1
    w.add_image(img, colormap=color, blending='additive')

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


DiGraph with 1002 nodes and 1001 edges
DiGraph with 426 nodes and 425 edges
DiGraph with 659 nodes and 658 edges
DiGraph with 129 nodes and 128 edges
DiGraph with 1494 nodes and 1493 edges
DiGraph with 53 nodes and 52 edges
DiGraph with 434 nodes and 433 edges
DiGraph with 437 nodes and 436 edges
DiGraph with 513 nodes and 512 edges
DiGraph with 26 nodes and 25 edges
DiGraph with 77 nodes and 76 edges
DiGraph with 687 nodes and 686 edges
DiGraph with 18 nodes and 17 edges
DiGraph with 2003 nodes and 2002 edges
DiGraph with 47 nodes and 46 edges
DiGraph with 65 nodes and 64 edges
DiGraph with 36 nodes and 35 edges
DiGraph with 894 nodes and 893 edges
DiGraph with 15 nodes and 14 edges
DiGraph with 30 nodes and 29 edges
DiGraph with 407 nodes and 406 edges
DiGraph with 317 nodes and 316 edges
DiGraph with 1137 nodes and 1136 edges
DiGraph with 209 nodes and 208 edges


In [88]:
roots_arr = np.array(list(roots))
roots_arr

array([[ 23, 135, 194],
       [ 20, 132, 173],
       [ 19, 159, 170],
       [ 29, 170, 182],
       [ 20, 159, 169],
       [ 21, 156, 167],
       [ 28, 170, 189],
       [ 22, 167, 195],
       [ 22, 130, 176],
       [ 22, 145, 170],
       [ 20, 160, 170],
       [ 27, 173, 187],
       [ 18, 153, 172],
       [ 23, 151, 167],
       [ 21, 144, 170],
       [ 21, 158, 169],
       [ 23, 147, 169],
       [ 21, 159, 170],
       [ 19, 159, 171],
       [ 30, 166, 181],
       [ 22, 149, 166],
       [ 29, 161, 183],
       [ 27, 170, 190],
       [ 22, 148, 167]])

In [89]:
bunches = []
min_dist = 4

for root in roots:
    roots_dists = np.linalg.norm(root - roots_arr, axis=-1)
    for r, d in zip(roots_arr[roots_dists < min_dist], roots_dists[roots_dists < min_dist]):
        print(root , r, d)
    neighbours = [tuple(r) for r in roots_arr[roots_dists < min_dist]]
    STOP=False
    for bunch in bunches:
        for n in neighbours:
            if n in bunch:
                bunch.update(neighbours)
                STOP = True
                continue
        
        if STOP:
            break
    if not STOP:
        bunches.append(set(neighbours))

(23, 135, 194) [ 23 135 194] 0.0
(20, 132, 173) [ 20 132 173] 0.0
(19, 159, 170) [ 19 159 170] 0.0
(19, 159, 170) [ 20 159 169] 1.4142135623730951
(19, 159, 170) [ 20 160 170] 1.4142135623730951
(19, 159, 170) [ 21 158 169] 2.449489742783178
(19, 159, 170) [ 21 159 170] 2.0
(19, 159, 170) [ 19 159 171] 1.0
(29, 170, 182) [ 29 170 182] 0.0
(20, 159, 169) [ 19 159 170] 1.4142135623730951
(20, 159, 169) [ 20 159 169] 0.0
(20, 159, 169) [ 21 156 167] 3.7416573867739413
(20, 159, 169) [ 20 160 170] 1.4142135623730951
(20, 159, 169) [ 21 158 169] 1.4142135623730951
(20, 159, 169) [ 21 159 170] 1.4142135623730951
(20, 159, 169) [ 19 159 171] 2.23606797749979
(21, 156, 167) [ 20 159 169] 3.7416573867739413
(21, 156, 167) [ 21 156 167] 0.0
(21, 156, 167) [ 21 158 169] 2.8284271247461903
(28, 170, 189) [ 28 170 189] 0.0
(28, 170, 189) [ 27 173 187] 3.7416573867739413
(28, 170, 189) [ 27 170 190] 1.4142135623730951
(22, 167, 195) [ 22 167 195] 0.0
(22, 130, 176) [ 22 130 176] 0.0
(22, 145, 170) [

In [90]:
bunches

[{(23, 135, 194)},
 {(20, 132, 173)},
 {(19, 159, 170),
  (19, 159, 171),
  (20, 159, 169),
  (20, 160, 170),
  (21, 156, 167),
  (21, 158, 169),
  (21, 159, 170)},
 {(29, 170, 182)},
 {(27, 170, 190), (27, 173, 187), (28, 170, 189)},
 {(22, 167, 195)},
 {(22, 130, 176)},
 {(21, 144, 170),
  (22, 145, 170),
  (22, 148, 167),
  (22, 149, 166),
  (23, 147, 169),
  (23, 151, 167)},
 {(18, 153, 172)},
 {(22, 148, 167), (22, 149, 166), (23, 151, 167)},
 {(30, 166, 181)},
 {(29, 161, 183)}]

In [91]:
set2del = []

for i, cur_bunch in enumerate(bunches[:-1]):
    FOUND = False
    for node in cur_bunch:
        for bunch in bunches[i+1:]:
            if node in bunch:
                bunch.update(cur_bunch)
                set2del.append(i)
                FOUND = True
                break
        if FOUND:
            break


In [92]:
bunches.pop(*set2del)

{(21, 144, 170),
 (22, 145, 170),
 (22, 148, 167),
 (22, 149, 166),
 (23, 147, 169),
 (23, 151, 167)}

In [93]:
bunches

[{(23, 135, 194)},
 {(20, 132, 173)},
 {(19, 159, 170),
  (19, 159, 171),
  (20, 159, 169),
  (20, 160, 170),
  (21, 156, 167),
  (21, 158, 169),
  (21, 159, 170)},
 {(29, 170, 182)},
 {(27, 170, 190), (27, 173, 187), (28, 170, 189)},
 {(22, 167, 195)},
 {(22, 130, 176)},
 {(18, 153, 172)},
 {(21, 144, 170),
  (22, 145, 170),
  (22, 148, 167),
  (22, 149, 166),
  (23, 147, 169),
  (23, 151, 167)},
 {(30, 166, 181)},
 {(29, 161, 183)}]

In [94]:
w = napari.view_image(data.image, opacity=0.5)
pos = {node: node for node in data.graph.nodes}
colors = ['blue', 'red', 'yellow', 'cyan', 'green', 'magenta', 'bop orange']


for bunch, color in zip(bunches, itt.cycle(colors)):
    img = np.zeros(data.image.shape)
    for root in bunch:
        points = draw_nodes(pos, branches[root].nodes)
        img[points[:, 0], points[:, 1], points[:, 2]] = 1
    w.add_image(img, colormap=color, blending='additive')

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [95]:
bunch = bunches[2]

In [96]:
max_mass = 0
main_branch_root = None
for branch_root in bunch:
    branch = branches[branch_root]
    branch_mass = len(branch.nodes)
    if branch_mass > max_mass:
        main_branch_root = branch_root
        main_branch = branches[main_branch_root]

In [97]:
points = draw_nodes(pos, main_branch.nodes)
main_branch_coords = np.array([points[:, 0], points[:, 1], points[:, 2]])

for branch_root in bunch:
    branch = branches[branch_root]
    points = draw_nodes(pos, branch.nodes)
    coords = np.array([points[:, 0], points[:, 1], points[:, 2]])
    count = min(len(branch.nodes), len(main_branch.nodes))
    dists = np.linalg.norm(coords[:, -count:] - main_branch_coords[:, -count:], axis=0)

#     print(dists)

In [98]:
branch = branches[(19, 159, 170)]

In [99]:
import astro_graph

In [100]:
astroGraph = astro_graph.AstroGraph(branch)

In [101]:
paths = list(astroGraph.graph_to_paths().values())[0]
# points = draw_nodes(pos, paths)

In [102]:
list(astroGraph.graph_to_paths().values())

[[[(19, 160, 169),
   (19, 161, 168),
   (19, 162, 167),
   (19, 163, 166),
   (19, 164, 165),
   (19, 165, 164),
   (19, 166, 163),
   (18, 167, 162),
   (18, 168, 161),
   (18, 169, 160),
   (18, 170, 160),
   (17, 171, 160)],
  [(17, 171, 160),
   (17, 172, 160),
   (17, 173, 159),
   (16, 174, 158),
   (15, 175, 157)],
  [(15, 175, 157),
   (15, 176, 156),
   (15, 177, 155),
   (15, 178, 154),
   (15, 179, 153),
   (15, 180, 152),
   (15, 181, 151)],
  [(15, 181, 151),
   (15, 181, 150),
   (14, 182, 149),
   (14, 182, 148),
   (14, 182, 147),
   (14, 182, 146),
   (14, 182, 145),
   (14, 182, 144),
   (14, 182, 143),
   (14, 181, 142),
   (14, 180, 141),
   (13, 179, 140),
   (13, 178, 139),
   (13, 177, 138),
   (13, 176, 137),
   (13, 175, 136),
   (13, 174, 135)],
  [(13, 174, 135), (13, 173, 134)],
  [(13, 173, 134), (13, 174, 134), (13, 173, 133), (13, 172, 132)],
  [(13, 172, 132),
   (13, 171, 131),
   (13, 170, 130),
   (13, 169, 129),
   (13, 169, 128)],
  [(13, 169, 128)

In [103]:
# w = napari.view_image(data.image, opacity=0.5)
# pos = {node: node for node in data.graph.nodes}

# colors = ['blue', 'red', 'yellow', 'cyan', 'green', 'magenta', 'bop orange']
# for path, color in zip(list(astroGraph.graph_to_paths().values())[0], itt.cycle(colors)):
#     img = np.zeros(data.image.shape)
#     points = draw_nodes(pos, path)
#     img[points[:, 0], points[:, 1], points[:, 2]] = 1
#     w.add_image(img, colormap=color, blending='additive')

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [104]:
w = napari.view_image(data.image, opacity=0.5)
pos = {node: node for node in data.graph.nodes}

for branch, color in zip(bunch, itt.cycle(colors)):
    img = np.zeros(data.image.shape)
    astroGraph = astro_graph.AstroGraph(branches[branch])
    path = list(astroGraph.graph_to_paths().values())[0][0]
    points = draw_nodes(pos, path)
    
    img[points[:, 0], points[:, 1], points[:, 2]] = 1
    w.add_image(img, colormap=color, blending='additive')    

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [106]:
# Adding main branch
img = np.zeros(data.image.shape)
astroGraph = astro_graph.AstroGraph(branches[main_branch_root])
path = list(astroGraph.graph_to_paths().values())[0][0]
points = draw_nodes(pos, path)

img[points[:, 0], points[:, 1], points[:, 2]] = 1
w.add_image(img, colormap='gray', blending='additive') 

<Image layer 'img [7]' at 0x7f504150d270>

In [109]:
main_branch_graph = astro_graph.AstroGraph(main_branch)
main_branch_path = [main_branch_root] + list(main_branch_graph.graph_to_paths().values())[0][0]
main_branch_points = draw_nodes(pos, main_branch_path)
main_branch_coords = np.array([main_branch_points[:, 0], main_branch_points[:, 1], main_branch_points[:, 2]])

for branch_root in bunch:
    branch = branches[branch_root]
    astroGraph = astro_graph.AstroGraph(branch)
    path = [branch_root] + list(astroGraph.graph_to_paths().values())[0][0]
    points = draw_nodes(pos, path)
#     print(points)
#     coords = np.array([points[:, 0], points[:, 1], points[:, 2]])
#     print(points[:count], main_branch_points[:count])
    count = min(len(points), len(main_branch_points))
    dists = np.linalg.norm(points[:count] - main_branch_points[:count], axis=-1)
    
    for p, mbp, d in zip(points[:count], main_branch_points[:count], dists):
        if d <= dists[0]:
            data.graph.remove_node(tuple(p))
    else:
        cur_p = p
        prev_p = p
        end_p = mbp
        azi = np.array([*np.sign(end_p - cur_p)])
        while tuple(cur_p) != tuple(end_p):
            cur_p = np.clip(cur_p + azi, np.min([p, mbp], axis=0), np.max([p, mbp], axis=0))
            data.graph.add_edge(tuple(prev_p), tuple(cur_p))
            prev_p = cur_p

In [80]:
ep = mbp + [2,-2,2]
# np.min([p, mbp+[2,-2,2]], axis=0)


In [81]:
np.clip(p+np.array([*np.sign(ep - p)]), np.min([p, ep], axis=0), np.max([p, ep], axis=0))

array([ 28, 131, 142])

In [111]:
w =  napari.view_image(data.image, ndisplay=3, opacity=0.5)
vis.view_graph_as_colored_image(data.graph, data.parameters['shape'], w);

/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [41]:
for branch_root in bunch:
    branch = branches[branch_root]
    astroGraph = astro_graph.AstroGraph(branch)
    path = [branch_root] + list(astroGraph.graph_to_paths().values())[0][0]
    print(branch_root, path)

(21, 144, 170) [(21, 144, 170), (21, 144, 169), (21, 144, 168), (21, 144, 167), (21, 144, 166), (21, 144, 165), (21, 144, 164), (21, 144, 163), (21, 144, 162), (21, 144, 161), (21, 144, 160), (21, 144, 159), (22, 143, 158), (22, 142, 157), (22, 141, 156), (22, 140, 155), (22, 139, 154), (22, 138, 153), (22, 137, 153), (22, 136, 153), (22, 135, 153), (22, 134, 154), (22, 133, 155), (22, 132, 155), (23, 131, 154), (23, 130, 153), (23, 129, 152), (22, 128, 151), (22, 129, 150), (21, 130, 149), (20, 131, 150), (19, 132, 149), (18, 133, 148), (17, 134, 147), (16, 135, 146), (16, 136, 145), (16, 136, 144), (17, 137, 143), (17, 138, 142), (16, 139, 141), (15, 139, 140), (15, 138, 139), (15, 137, 139), (15, 136, 139), (15, 135, 139), (15, 134, 139), (15, 133, 139)]
(23, 147, 169) [(23, 147, 169), (23, 147, 168), (23, 147, 167), (23, 147, 166), (23, 147, 165), (23, 147, 164), (23, 147, 163), (23, 147, 162), (23, 147, 161), (23, 147, 160), (23, 147, 159), (23, 147, 158), (23, 147, 157), (23, 146